In [3]:
import pandas as pd
import numpy as np
from typing import Callable
from ICARUS.Database.db import DB
import os
import matplotlib.pyplot as plt
from typing import Any
from numpy import dtype, floating, ndarray
from scipy.interpolate import CubicSpline

plt.ion()
import time

In [4]:
t0 = 0
tend = 60
x0 =  [0, 20, np.deg2rad(0.1)]
operating_floor = 12.5
elevator_max_deflection = 15
v_mag_0 = 17.38

In [6]:
engine_dir = "Data/Engine/Motor_1/"
prop_dfs: dict[str, pd.DataFrame] = {}
for prop_data in os.listdir(engine_dir):
    if prop_data.endswith(".csv"):
        prop_name = prop_data[:-4]
        prop_dfs[prop_name] = pd.read_csv(engine_dir + prop_data)

In [7]:
prop_dfs.keys(  )
motor = prop_dfs['12x6E'].sort_values(by=['Airspeed [m/s]'])
motor["Thrust [N]"] = motor["Thrust [g]"] * 9.81 / 1000

In [8]:
W = 4
gamma = 3
Cl_somewhat = 0.5
CD_0 = 0.05


In [9]:
def interpolate_polars(aoa, cldata):
    cl =  np.interp(aoa, cldata['AoA'], cldata['CL_2D'])
    cd = np.interp(aoa, cldata['AoA'], cldata['CD_2D'])
    cm = np.interp(aoa, cldata['AoA'], cldata['Cm_2D'])
    return cl, cd, cm

def interpolate_thrust(velocity, motor):
    throtle_levels = motor['Throttle [%]'].unique()
    min_throtle = motor[motor['Throttle [%]'] == min(throtle_levels)]
    max_throtle = motor[motor['Throttle [%]'] == max(throtle_levels)]

    min_thrust = np.interp(velocity, min_throtle['Airspeed [m/s]'], min_throtle['Thrust [N]'])
    max_thrust = np.interp(velocity, max_throtle['Airspeed [m/s]'], max_throtle['Thrust [N]'])

    return (min_thrust, max_thrust)

def get_lift_drag_torque(plane: Airplane, velocity, aoa, cldata):
    aoa = np.rad2deg(aoa)
    cl , cd, cm= interpolate_polars(aoa, cldata[['CL_2D', "CD_2D","Cm_2D", "AoA"]])
    density = 1.225
    lift = cl * 0.5 * density * velocity**2 * plane.S
    drag = cd * 0.5 * density * velocity**2 * plane.S
    torque = cm * 0.5 * density * velocity**2 * plane.S * plane.mean_aerodynamic_chord

    return lift, drag, torque

def get_elevator_lift_over_cl(elevator, velocity, aoa):
    aoa = np.rad2deg(aoa)
    density = 1.225
    lift_over_cl =  np.pi * density * velocity**2 * elevator.S

    return lift_over_cl

def get_thrust(alpha,velocity, thrust):
    thrust = interpolate_thrust(velocity, thrust)
    thrust_x = thrust * np.cos(alpha)
    thrust_y = thrust * np.sin(alpha)
    return thrust_x, thrust_y